<a href="https://colab.research.google.com/github/Stephanieessmak/database/blob/main/Flat_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from pathlib import Path
import datetime
import io
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from itertools import repeat
import warnings
from google.colab import drive
drive.mount('/content/drive')
warnings.filterwarnings('ignore') 

Mounted at /content/drive


In [ ]:
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
json = '/content/drive/MyDrive/Essmak Data Work/Requirements/Credentials.json'
credentials = Credentials.from_service_account_file(json, scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

In [ ]:
df = gc.open_by_url("https://docs.google.com/spreadsheets/d/1I8NdsQyAWgI7p237Hj7OsBou56IA1wYUHrXvuOFRj6M/edit#gid=67394343")
crm = df.worksheet('CRM')
crm=pd.DataFrame(crm.get_all_records()) 

fc = df.worksheet('FirstCry')
fc=pd.DataFrame(fc.get_all_records()) 

mw = df.worksheet('Mumzworld')
mw=pd.DataFrame(mw.get_all_records())

In [ ]:
# for crm - product category changed to product name
fc=fc[['Order Date', 'Total USD', "Order Id's", 'Qty', 'Style Code','Country' ,'day', 'month', 'year',"Icon Category", "Product name ","Product Name FYI",
          "Product_Category","Product SKU","Product","Cost Price",  'Franchise fees_percent',	'Franchise fees_2_percent',  "Franchise_amount","Total Cost", "Profit"]]
mw = mw[['Date',  'Total USD','PO Number', 'Order Qty', 'MW SKU','Country' ,'day', 'month', 'year',  "Icon Category", "Product name ","Product Name FYI",
         "Product_Category","Product SKU","Product","Cost Price",'Franchise fees_percent',	'Franchise fees_2_percent', "Franchise_amount","Total Cost", "Profit"]]
crm = crm[['Order Date', 'Total', 'Order ID', 'Quantity', 'SKU Name', 'Shipping Country', 'day', 'month', 'year', "Icon Category", "Product name ","Product Name FYI",
          "Product_Category","Product SKU","Product","Cost Price",'Franchise fees_percent',	'Franchise fees_2_percent', "Franchise_amount","Total Cost", "Profit" ]] 

In [ ]:
column_indices = [0,1,2,3,4,5,6,7,8,9,10, 11,12,13,14,15,16,17,18,19,20]
new_names = ['Order Date', 'Total_amount', 'Order ID', 'Quantity', 'Product SKU Source','Country' ,'day', 'month', 'year', "Icon Category", "Product name ","Product Name FYI",
             "Product_Category","Product SKU","Product","Cost Price",'Franchise fees_percent',	'Franchise fees_2_percent', "Franchise_amount","Total Cost", "Profit"]
old_names = fc.columns[column_indices]
fc.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [ ]:
column_indices = [0,1,2,3,4,5,6,7,8,9,10, 11,12,13,14,15,16,17,18,19,20]
new_names = ['Order Date', 'Total_amount', 'Order ID', 'Quantity', 'Product SKU Source', 'Country','day', 'month', 'year', "Icon Category", "Product name ","Product Name FYI",
             "Product_Category","Product SKU","Product","Cost Price",'Franchise fees_percent',	'Franchise fees_2_percent', "Franchise_amount","Total Cost", "Profit"]
old_names = mw.columns[column_indices]
mw.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [ ]:
column_indices = [0,1,2,3,4,5,6,7,8,9,10, 11,12,13,14,15,16,17,18,19,20]
new_names = ['Order Date', 'Total_amount', 'Order ID', 'Quantity', 'Product SKU Source', 'Country','day', 'month', 'year', "Icon Category", "Product name ","Product Name FYI",
             "Product_Category","Product SKU","Product","Cost Price",'Franchise fees_percent',	'Franchise fees_2_percent', "Franchise_amount","Total Cost", "Profit"]
old_names = crm.columns[column_indices]
crm.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [ ]:
Source = 'FirstCry'
fc['Data_source'] = Source 
Source1 = 'Mumzworld'
mw['Data_source'] = Source1 
Source2 = 'CRM'
crm['Data_source'] = Source2

In [ ]:
ft = crm.append(fc, ignore_index=True) 

In [ ]:
ft = ft.append(mw, ignore_index=True )

In [ ]:
ft['Order ID']= ft['Data_source'].astype(str) +"-"+ ft["Order ID"].astype(str)

In [ ]:
ft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33328 entries, 0 to 33327
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Order Date                33328 non-null  object 
 1   Total_amount              33328 non-null  object 
 2   Order ID                  33328 non-null  object 
 3   Quantity                  33328 non-null  int64  
 4   Product SKU Source        33328 non-null  object 
 5   Country                   33328 non-null  object 
 6   day                       33328 non-null  int64  
 7   month                     33328 non-null  int64  
 8   year                      33328 non-null  int64  
 9   Icon Category             33328 non-null  object 
 10  Product name              33328 non-null  object 
 11  Product Name FYI          33328 non-null  object 
 12  Product_Category          33328 non-null  object 
 13  Product SKU               33328 non-null  object 
 14  Produc

In [ ]:
gs = gc.open_by_url('https://docs.google.com/spreadsheets/d/1I8NdsQyAWgI7p237Hj7OsBou56IA1wYUHrXvuOFRj6M/edit#gid=67394343')
worksheet2 = gs.worksheet('Flat_Table')
worksheet2.clear()
set_with_dataframe(worksheet=worksheet2, dataframe=ft, include_index=False,include_column_header=True, resize=True) 